In [ ]:
sequence1 = "ATCGTACG"
sequence2 = "GTTACGTA"
sequence2 = "CGTACGTA"


In [ ]:
import numpy as np

def dna_to_numeric(sequence):
    """Convert a DNA sequence to a numeric sequence."""
    mapping = {'A': 1, 'T': 2, 'C': 3, 'G': 4}
    return np.array([mapping[nucleotide] for nucleotide in sequence])

def pearson_correlation(sequence1, sequence2):
    """Calculate the Pearson correlation coefficient between two DNA sequences."""
    num_seq1 = dna_to_numeric(sequence1)
    num_seq2 = dna_to_numeric(sequence2)
    return np.corrcoef(num_seq1, num_seq2)[0, 1]

# Example sequences
sequence1 = "ATCGTACG"
sequence2 = "GTTACGTA"

# Calculate Pearson correlation coefficient
correlation = pearson_correlation(sequence1, sequence2)
correlation

-0.9559937503105924

In [ ]:
# Define example sequences
sequences = ["ATCGTACG", "GTTACGTA", "CGTACGTA"]

# Construct the distance matrix
num_sequences = len(sequences)
distance_matrix = np.zeros((num_sequences, num_sequences))

for i in range(num_sequences):
    for j in range(num_sequences):
        if i != j:
            # We use 1 minus absolute value of correlation to ensure non-negative distances
            # and that a higher correlation results in a shorter distance.
            distance_matrix[i, j] = 1 - abs(pearson_correlation(sequences[i], sequences[j]))

distance_matrix

array([[0.        , 0.04400625, 0.1       ],
       [0.04400625, 0.        , 0.24526809],
       [0.1       , 0.24526809, 0.        ]])

In [ ]:
def construct_quadratic_form(distance_matrix):
    """Construct the quadratic form Q for the given distance matrix."""
    num_sequences = len(distance_matrix)

    # Initialize Q as a dictionary {(i, j): value}
    Q = {}

    # Populate Q using the distance matrix
    for i in range(num_sequences):
        for j in range(num_sequences):
            for k in range(num_sequences):
                for l in range(num_sequences):
                    if i < k:
                        key = (i * num_sequences + j, k * num_sequences + l)
                        if j == l:
                            Q[key] = Q.get(key, 0) + distance_matrix[i][k]
                        if j != l and i == k:
                            Q[key] = Q.get(key, 0) + 2 * distance_matrix[i][j]

    # We want to minimize the objective function, so we negate Q
    for key in Q:
        Q[key] = -Q[key]

    return Q

# Construct the quadratic form Q based on the distance matrix
Q = construct_quadratic_form(distance_matrix)
Q

{(0, 3): -0.044006249689407584,
 (0, 6): -0.09999999999999998,
 (1, 4): -0.044006249689407584,
 (1, 7): -0.09999999999999998,
 (2, 5): -0.044006249689407584,
 (2, 8): -0.09999999999999998,
 (3, 6): -0.2452680918600585,
 (4, 7): -0.2452680918600585,
 (5, 8): -0.2452680918600585}

In [ ]:
import math
import random

def cost(solution, distance_matrix):
    """Compute the cost of a given solution based on the distance matrix."""
    return sum(distance_matrix[solution[i]][solution[i + 1]] for i in range(len(solution) - 1))

def simulated_annealing(distance_matrix, initial_temperature, cooling_rate, num_iterations):
    """Approximate a solution using the Simulated Annealing algorithm."""
    num_sequences = len(distance_matrix)

    # Start with a random solution
    current_solution = list(range(num_sequences))
    random.shuffle(current_solution)
    current_cost = cost(current_solution, distance_matrix)

    best_solution = current_solution[:]
    best_cost = current_cost

    temperature = initial_temperature

    for iteration in range(num_iterations):
        # Generate a neighboring solution by swapping two sequences
        i, j = random.sample(range(num_sequences), 2)
        neighbor_solution = current_solution[:]
        neighbor_solution[i], neighbor_solution[j] = neighbor_solution[j], neighbor_solution[i]
        neighbor_cost = cost(neighbor_solution, distance_matrix)

        # Calculate cost difference
        cost_diff = neighbor_cost - current_cost

        # Determine if neighbor solution should be accepted
        if cost_diff < 0 or random.random() < math.exp(-cost_diff / temperature):
            current_solution, current_cost = neighbor_solution, neighbor_cost

            # Update best solution
            if current_cost < best_cost:
                best_solution, best_cost = current_solution, current_cost

        # Reduce the temperature
        temperature *= cooling_rate

    return best_solution, best_cost

# Parameters for the Simulated Annealing algorithm
initial_temperature = 100
cooling_rate = 0.995
num_iterations = 10000

# Run the Simulated Annealing algorithm
solution, solution_cost = simulated_annealing(distance_matrix, initial_temperature, cooling_rate, num_iterations)
solution, solution_cost

([2, 0, 1], 0.14400624968940756)

improved simulated annealing

In [ ]:
def advanced_neighbor(solution):
    """Generate a neighboring solution using an advanced strategy."""
    # With a 50% probability, use the swap strategy
    if random.random() < 0.5:
        i, j = random.sample(range(len(solution)), 2)
        neighbor_solution = solution[:]
        neighbor_solution[i], neighbor_solution[j] = neighbor_solution[j], neighbor_solution[i]
    # Otherwise, use the reverse subsequence strategy
    else:
        i, j = sorted(random.sample(range(len(solution)), 2))
        neighbor_solution = solution[:i] + solution[i:j+1][::-1] + solution[j+1:]
    return neighbor_solution

def improved_simulated_annealing(distance_matrix, initial_temperature, cooling_rate, num_iterations, num_runs):
    """An improved version of the Simulated Annealing algorithm."""
    best_solution = None
    best_cost = float('inf')

    for run in range(num_runs):
        # Initial random solution
        current_solution = list(range(len(distance_matrix)))
        random.shuffle(current_solution)
        current_cost = cost(current_solution, distance_matrix)

        temperature = initial_temperature

        for iteration in range(num_iterations):
            # Generate a neighboring solution using the advanced strategy
            neighbor_solution = advanced_neighbor(current_solution)
            neighbor_cost = cost(neighbor_solution, distance_matrix)

            # Calculate cost difference
            cost_diff = neighbor_cost - current_cost

            # Determine if neighbor solution should be accepted
            if cost_diff < 0 or random.random() < math.exp(-cost_diff / temperature):
                current_solution, current_cost = neighbor_solution, neighbor_cost

            # Reduce the temperature
            temperature *= cooling_rate

        # Update the best solution from all runs
        if current_cost < best_cost:
            best_solution, best_cost = current_solution, current_cost

    return best_solution, best_cost

# Parameters for the improved Simulated Annealing algorithm
num_runs = 100

# Run the improved Simulated Annealing algorithm
improved_solution, improved_solution_cost = improved_simulated_annealing(
    distance_matrix, initial_temperature, cooling_rate, num_iterations, num_runs)
improved_solution, improved_solution_cost

([1, 0, 2], 0.14400624968940756)